# Setup

Our targets come from the Penn World Tables and the OECD for the years 1995 to 2015.

In [16]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

In [17]:
countries = ['us','fr','it','dk','sp','nl','se','de']
countries = [c.upper() for c in countries]
cnames = ['United States','France','Italy','Denmark','Spain','Netherlands','Sweden','Germany']
co = dict(zip(countries,cnames))
ce = dict(zip(cnames,countries))
data = '../data_sources/'

We use Penn World Tables (https://www.rug.nl/ggdc/productivity/pwt/) for the capital to income ratio,  $\frac{k}{y}$, consumption to income,  $\frac{c}{y}$, and for relative GDP  $\frac{y}{y_{US}}$.

In [18]:
penn = pd.read_excel(data+'pwt/pwt90.xlsx',sheet_name='Data')

Select years and countries

In [19]:
years = range(1995,2016,1)

In [20]:
penn = penn[(penn['year'].isin(years)) & (penn['country'].isin(cnames))]

In [21]:
penn.head()

,countrycode,country,currency_unit,year,rgdpe,rgdpo,pop,emp,avh,hc,...,csh_g,csh_x,csh_m,csh_r,pl_c,pl_i,pl_g,pl_x,pl_m,pl_k
3035,DEU,Germany,Euro,1995,2444280.75,2376434.00,81.612900,37.939034,1527.978292,3.497632,...,0.135220,0.348669,-0.361674,0.000000e+00,1.033949,0.952057,1.559026,0.694944,0.655703,1.145803
3036,DEU,Germany,Euro,1996,2474957.50,2409760.75,81.870772,37.959774,1510.495404,3.511262,...,0.134964,0.346536,-0.353329,1.859867e-16,0.976098,0.934430,1.514511,0.696348,0.658500,1.094040
3037,DEU,Germany,Euro,1997,2565733.00,2481533.50,81.993831,37.939022,1499.644241,3.524946,...,0.136400,0.346886,-0.342708,0.000000e+00,0.834261,0.826947,1.250930,0.656168,0.633227,0.969747
3038,DEU,Germany,Euro,1998,2670308.75,2555663.00,82.010184,38.400684,1493.581899,3.538684,...,0.136556,0.374267,-0.359334,-2.075864e-16,0.808360,0.836381,1.193275,0.612878,0.606401,0.974770
3039,DEU,Germany,Euro,1999,2768837.75,2641263.50,81.965830,38.979191,1478.722041,3.552474,...,0.139956,0.365609,-0.357325,0.000000e+00,0.758368,0.812084,1.107785,0.606163,0.603862,0.954278


We now compute variables

# Construct Variables

In [22]:
penn = penn[['country','year','pop','rconna','rgdpna','rkna']]
penn['year'] = penn['year'].astype(str)

We use real consumption at 2011 level National prices (in millions, 2011 US).

In [23]:
penn['csy'] = penn['rconna']/penn['rgdpna']

We use the capital stock at constant 2011 National prices (in millions, 2011 US)

In [24]:
penn['ksy'] = penn['rkna']/penn['rgdpna']

We compute GDP per capita using real GDP at constant 2011 prices (in millions 2011 US) and divide by total population (in millions).

In [25]:
penn['rgdpnap'] = penn['rgdpna']/penn['pop']

We compute deviations relative to US.

In [26]:
temp = penn[['rgdpnap','country','year']]
temp = temp.set_index(['year','country'])
temp = temp.unstack()
for c in cnames[1:]:
    temp['rgdpnap'][c] = temp['rgdpnap'][c]/temp['rgdpnap']['United States']
temp['rgdpnap']['United States'] = temp['rgdpnap']['United States']/temp['rgdpnap']['United States']
temp = temp.stack()
temp.columns = ['ys']
ys = temp.reset_index().set_index(['country','year']).sort_index()
penn = penn.set_index(['country','year'])
penn = penn.merge(ys,left_index=True,right_index=True)
penn = penn[['csy','ksy','ys']]

We have the following moments across countries and years

In [27]:
penn.describe()

,csy,ksy,ys
count,160.000000,160.000000,160.000000
mean,0.744910,4.226871,0.814297
std,0.047945,0.650458,0.108000
min,0.636059,2.962467,0.599292
25%,0.710768,3.902014,0.735718
50%,0.751858,4.240006,0.820280
75%,0.770197,4.552309,0.883126
max,0.851606,5.936691,1.000000


In [28]:
penn.head()

csy       ksy        ys
country year                              
Germany 1995  0.770485  3.612009  0.864700
        1996  0.775610  3.668945  0.847332
        1997  0.767639  3.685918  0.835105
        1998  0.763689  3.702503  0.825562
        1999  0.765286  3.723471  0.814552

# GDP Share of Expenditures

We take the GDP share of medical expenditures (all financing sources) from OECD Health Data 2018. We process the table downloaded directly from OECD.

In [29]:
labels = ['country','unit','scrap']
for t in range(1995,2016):
    labels.append(str(t))
oecd = pd.read_excel(data+'oecd/oecd-shares-1995-2015.xlsx',skiprows=7,skipfooter=3)
oecd.columns = labels
oecd = oecd.drop(columns=['scrap','unit'])
oecd = oecd.set_index('country')
oecd = oecd.stack().to_frame()
oecd.columns = ['s']
oecd['s'] = oecd['s']/100.0
oecd.index.names = ['country','year']

In [30]:
oecd.head()

s
country year         
Denmark 1995  0.07771
        1996  0.07839
        1997  0.07750
        1998  0.07552
        1999  0.08342

We merge back to Penn Tables.

In [31]:
penn = penn.merge(oecd,left_index=True,right_index=True)
penn = penn.reset_index()

In [32]:
penn.head()

,country,year,csy,ksy,ys,s
0,Germany,1995,0.770485,3.612009,0.864700,0.09500
1,Germany,1996,0.775610,3.668945,0.847332,0.09785
2,Germany,1997,0.767639,3.685918,0.835105,0.09664
3,Germany,1998,0.763689,3.702503,0.825562,0.09687
4,Germany,1999,0.765286,3.723471,0.814552,0.09768


# Statistics

We compute consumption as a share of GDP consistent with the model. We only need this statistic for the U.S. as it is only used for that country. WE substract government consumption from Penn World Table consumption but add health spending which is public, Medicare and Medicaid. The first step is to get government consumption as a share of GDP. We use Fred for this (https://fred.stlouisfed.org/series/DPCERE1Q156NBEA).We keep the value for 2005. 

In [41]:
fred = pd.read_excel('../data_sources/fred/gsy.xls',sheet_name='data')
gsy = fred.loc[fred.year<2016,'gsy'].mean()/100.0
gsy

0.18980952380952382

In [42]:
fred

,year,gsy
0,1995,18.975
1,1996,18.500
2,1997,18.050
3,1998,17.775
4,1999,17.850
5,2000,17.825
6,2001,18.425
7,2002,19.125
8,2003,19.275
9,2004,19.150


We substract from public spending the part attributed to Medicare and Medicaid, which from National Health Accounts, represents 33% of all personal Health Expenditures.  

In [52]:
share_pub = 0.33

We then adjust the Penn World Numbers

In [53]:
penn.loc[penn.country=='United States','cnsy'] = penn.loc[penn.country=='United States','csy'] - (gsy - share_pub*penn.loc[penn.country=='United States','s'])

We compute means and standard errors by country over these years.

In [54]:
targets = penn[['cnsy','s','ys','ksy']].groupby(penn['country']).describe()
targets = targets.reset_index()
targets['co'] = targets['country'].replace(ce)
targets.set_index('co',inplace=True)

In [55]:
names = ['cnsy','s','ys','ksy']

# Saving Targets

We save means and standard error of the mean in moments directory.

In [56]:
means = targets.loc[:,targets.columns.get_level_values(1)=='mean']
means.columns = ['cshare','mshare','tfp','kshare']
means.loc[means.index.isin([c for c in countries if c!='US']),'cshare'] = np.nan
stds = targets.loc[:,targets.columns.get_level_values(1)=='std']/np.sqrt(2015-1995+1)
stds.columns = ['cshare','mshare','tfp','kshare']
stds.loc[stds.index.isin([c for c in countries if c!='US']),'cshare'] = np.nan
means.transpose()

/Users/loulou/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/loulou/.local/lib/python3.8/site-packages/pandas/core/indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


co,DK,FR,DE,IT,NL,SP,SE,US
cshare,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.682899
mshare,0.090629,0.103574,0.102286,0.081089,0.088282,0.078123,0.085785,0.144225
tfp,0.898205,0.735264,0.830156,0.739971,0.872562,0.640259,0.797962,1.000000
kshare,4.134086,4.426736,3.807190,5.116111,4.294942,4.827075,4.131106,3.077725


In [57]:
means.loc[['DK','SE','FR','IT','NL','SP','SE'],'mshare'].mean(),means.loc[['DK','SE','FR','IT','NL','SP','SE'],'tfp'].mean()


(0.0876095, 0.7831693755107656)

In [58]:
means.transpose().to_pickle('../estimation/moments/means_macro.pkl')


In [59]:
stds.transpose()

co,DK,FR,DE,IT,NL,SP,SE,US
cshare,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.017642
mshare,0.009913,0.007025,0.005096,0.007282,0.014508,0.009155,0.013101,0.016178
tfp,0.027095,0.012841,0.024683,0.046430,0.018157,0.021694,0.032258,0.000000
kshare,0.250125,0.199957,0.109525,0.444126,0.143179,0.532728,0.257594,0.074448


In [51]:
stds.transpose().to_pickle('../estimation/moments/stds_macro.pkl')